# SGAN:

In [7]:
!pip install Augmentor

# Importing Req Lib

In [57]:
import Augmentor
import os
import shutil
import random
import tensorflow as tf
import numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, roc_auc_score
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model


# Data loading 

In [11]:
#calling the dir
with_detected_dir = '/Users/hardikchhipa/Desktop/Data_Manipulations_Projects/Furcation detection/with defect'
without_detected_dir = '/Users/hardikchhipa/Desktop/Data_Manipulations_Projects/Furcation detection/without defect'

# Data Augmentation

In [12]:
target_count = 1000

# Current number of images
current_with_detected = 53
current_without_detected = 320

# Calculate the number of images to generate
images_to_generate_with = target_count - current_with_detected
images_to_generate_without = target_count - current_without_detected

In [14]:
# Augmentation for "with detected" images
p_with = Augmentor.Pipeline(with_detected_dir)
p_with.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
p_with.zoom_random(probability=0.5, percentage_area=0.8)
p_with.flip_left_right(probability=0.5)
p_with.flip_top_bottom(probability=0.3)
p_with.shear(probability=0.4, max_shear_left=5, max_shear_right=5)
p_with.random_contrast(probability=0.3, min_factor=0.8, max_factor=1.2)
p_with.random_brightness(probability=0.3, min_factor=0.8, max_factor=1.2)
p_with.sample(images_to_generate_with)  # Generate the required images
print(f"Generated {images_to_generate_with} images for 'with detected'.")

Initialised with 53 image(s) found.
Output directory set to /Users/hardikchhipa/Desktop/Data_Manipulations_Projects/Furcation detection/with defect/output.

Processing <PIL.Image.Image image mode=RGB size=2440x1292 at 0x1415A2150>: 100%|██████████| 947/947 [00:56<00:00, 16.87 Samples/s]                  

Generated 947 images for 'with detected'.


In [15]:
# Augmentation for "without detected" images
p_without = Augmentor.Pipeline(without_detected_dir)
p_without.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
p_without.zoom_random(probability=0.5, percentage_area=0.8)
p_without.flip_left_right(probability=0.5)
p_without.flip_top_bottom(probability=0.3)
p_without.shear(probability=0.4, max_shear_left=5, max_shear_right=5)
p_without.random_contrast(probability=0.3, min_factor=0.8, max_factor=1.2)
p_without.random_brightness(probability=0.3, min_factor=0.8, max_factor=1.2)
p_without.sample(images_to_generate_without)  # Generate the required images
print(f"Generated {images_to_generate_without} images for 'without detected'.")

Initialised with 320 image(s) found.
Output directory set to /Users/hardikchhipa/Desktop/Data_Manipulations_Projects/Furcation detection/without defect/output.

Processing <PIL.Image.Image image mode=RGB size=2440x1292 at 0x14156E420>: 100%|██████████| 680/680 [00:37<00:00, 18.04 Samples/s]                  

Generated 680 images for 'without detected'.


# Splitting Data

In [30]:
# Create new directories for train and validation datasets
base_dir = '/Users/hardikchhipa/Desktop/Data_Manipulations_Projects/Furcation detection/split_data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

train_with_dir = os.path.join(train_dir, 'with_defect')
train_without_dir = os.path.join(train_dir, 'without_defect')
val_with_dir = os.path.join(val_dir, 'with_defect')
val_without_dir = os.path.join(val_dir, 'without_defect')

os.makedirs(train_with_dir, exist_ok=True)
os.makedirs(train_without_dir, exist_ok=True)
os.makedirs(val_with_dir, exist_ok=True)
os.makedirs(val_without_dir, exist_ok=True)

In [31]:
# Split data function
def split_data(source_dir, train_dest, val_dest, split_ratio=0.8):
    all_files = os.listdir(source_dir)
    random.shuffle(all_files)
    split_index = int(len(all_files) * split_ratio)
    train_files = all_files[:split_index]
    val_files = all_files[split_index:]
    
    for file in train_files:
        shutil.copy(os.path.join(source_dir, file), os.path.join(train_dest, file))
    for file in val_files:
        shutil.copy(os.path.join(source_dir, file), os.path.join(val_dest, file))

In [32]:
split_data(with_detected_dir, train_with_dir, val_with_dir, split_ratio=0.8)
split_data(without_detected_dir, train_without_dir, val_without_dir, split_ratio=0.8)

print(f"Train directory: {train_dir}")
print(f"Validation directory: {val_dir}")

Train directory: /Users/hardikchhipa/Desktop/Data_Manipulations_Projects/Furcation detection/split_data/train
Validation directory: /Users/hardikchhipa/Desktop/Data_Manipulations_Projects/Furcation detection/split_data/val


In [63]:
train_dir = '/Users/hardikchhipa/Desktop/Data_Manipulations_Projects/Furcation detection/split_data/train'
val_dir = '/Users/hardikchhipa/Desktop/Data_Manipulations_Projects/Furcation detection/split_data/val'

# ImageDataGenerator for preprocessing
train_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
val_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

# Data loaders
train_data = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(1024, 1024), 
    batch_size=32, 
    class_mode='binary'
)
val_data = val_datagen.flow_from_directory(
    val_dir, 
    target_size=(1024, 1024), 
    batch_size=32, 
    class_mode='binary'
)

Found 1598 images belonging to 2 classes.
Found 402 images belonging to 2 classes.


# Transfer Learning Model(Densenet121)

In [66]:
# Load a pretrained DenseNet121 model
base_model = tf.keras.applications.DenseNet121(
    include_top=False,
    weights='imagenet',
    input_shape=(1024, 1024, 3)
)

# Freeze the base model for initial training
base_model.trainable = False

# Add custom layers on top
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# # Summary of the model
# model.summary()

In [67]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,  # Adjust epochs based on your resources
    verbose=1
)

Epoch 1/10


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


50/50 ━━━━━━━━━━━━━━━━━━━━ 1503s 30s/step - accuracy: 0.5690 - loss: 0.7246 - val_accuracy: 0.5970 - val_loss: 0.6374
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1419s 28s/step - accuracy: 0.6430 - loss: 0.6305 - val_accuracy: 0.6443 - val_loss: 0.6164
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 2256s 45s/step - accuracy: 0.6364 - loss: 0.6115 - val_accuracy: 0.7189 - val_loss: 0.5489
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 3190s 65s/step - accuracy: 0.6838 - loss: 0.5834 - val_accuracy: 0.7189 - val_loss: 0.5359
Epoch 5/10
 9/50 ━━━━━━━━━━━━━━━━━━━━ 41:18 60s/step - accuracy: 0.7266 - loss: 0.5600

Traceback (most recent call last):
Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
  File "/opt/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/p2/6mgfcw956d17xkqh172sdjk40000gn/T/ipykernel_57405/3156088862.py", line 1, in <module>
    history = model.fit(
              ^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/keras/src/utils/traceback_utils.py", line 117, in error_handler
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/keras/src/backend/tensorflow/trainer.py", line 318, in fit
    logs = self.train_function(iterator)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/tensorflow/python/util/tr

KeyboardInterrupt: 

In [65]:
for x_batch, y_batch in train_data:
    print("Batch shape:", x_batch.shape)
    break

Batch shape: (32, 1024, 1024, 3)


* * *
#### **Hardik Chhipa**  
BTech CSE | Aspiring Data Scientist  
Machine Learning & Deep Learning Enthusiast  
SRM University, Chennai

📧 [hardikchhipa28@gmail.com](mailto:hardikchhipa28@gmail.com)      📱 [+91 9460626737](tel:+919460626737)     🌐 [GitHub: FRAGGERR](https://github.com/FRAGGERR)    🔗 [LinkedIn: Hardik Chhipa](https://www.linkedin.com/in/hardik-chhipa-303040242/)

* * *

Sent with ❤️ from my Notebook